<a href="https://colab.research.google.com/github/Troyanovsky/Local-LLM-Comparison-Colab-UI/blob/main/dolphin_2_5_mixtral_8x7b_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## dolphin-2.5-mixtral-8x7b-GGUF WebUI
1. Run the following cell, takes ~5 min
(You may need to confirm to proceed by typing "Y")
2. Click the gradio link at the bottom
3. In Chat settings - Instruction Template: ChatML

```
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
```

Note:
- only run the cell with the quantization you need
- only part of the layers are loaded to GPU so the generation speed will be slow (if you have colab pro, you can adjust the gpu layers to a higher number for faster speed)

In [1]:
%cd /content
!apt-get -y install -qq aria2

!git clone -b V20231218 https://github.com/Troyanovsky/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install -U gradio==3.50.2

!pip uninstall -y llama-cpp-python -y
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

!pip uninstall flash-attn -y
!pip install --no-build-isolation flash-attn==2.3.4

/content
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.

Found existing installation: llama_cpp_python 0.2.23+cpuavx2
Uninstalling llama_cpp_python-0.2.23+cpuavx2:
  Successfully uninstalled llama_cpp_python-0.2.23+cpuavx2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 29.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.24-cp310-cp310-manylinux_2_35_x86_64.whl size=8156146 sha256=3e0c713576fd2bbacae4360fe135599d857bc2ea0ec294fc534d3675b4c0e969
  Stored in directory: /tmp/pip-ephem-wheel-cache-9tyqon6j/wheels/9b/5f/31/172f820a61d82eb7b057b7d05ff9a05345012066e8ae781788
Successfully built llama-cpp-python
Found existing installation: flash-attn 2.3.4
Uninstalling flash-attn-2.3.4:
  Successfully uninstalled flash-attn-2.3.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.5 MB/s eta 0:00:00
  Preparing metadata 

In [ ]:
# Q2_K, some on GPU, faster generation (1.68 T/s)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/resolve/main/dolphin-2.5-mixtral-8x7b.Q2_K.gguf?download=true -d /content/text-generation-webui/models/ -o dolphin-2.5-mixtral-8x7b.Q2_K.gguf

%cd /content/text-generation-webui
!python server.py --share --n-gpu-layers 20 --n_ctx 2048 --model dolphin-2.5-mixtral-8x7b.Q2_K.gguf

 *** Download Progress Summary as of Wed Dec 20 06:11:29 2023 *** 
=
[#41587c 12GiB/14GiB(88%) CN:16 DL:202MiB ETA:8s]
FILE: /content/text-generation-webui/models//dolphin-2.5-mixtral-8x7b.Q2_K.gguf
-


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
41587c|OK  |   220MiB/s|/content/text-generation-webui/models//dolphin-2.5-mixtral-8x7b.Q2_K.gguf

Status Legend:
(OK):download completed.
/content/text-generation-webui
2023-12-20 06:11:45 INFO:Loading dolphin-2.5-mixtral-8x7b.Q2_K.gguf...
2023-12-20 06:11:45 INFO:llama.cpp weights detected: models/dolphin-2.5-mixtral-8x7b.Q2_K.gguf
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   yes
ggml_init_cublas: CUDA_USE_TENSOR_CORES: no
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from models/dolphin-2.5-mixtral-8x7b.Q2_K.gguf (version GGUF V3 (latest))
llama

In [ ]:
# Q4_K_M, half on GPU, very slow generation
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/resolve/main/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf?download=true -d /content/text-generation-webui/models/ -o dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf

%cd /content/text-generation-webui
!python server.py --share --n-gpu-layers 12 --n_ctx 2048 --model dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf